### Questions
- What are the different state/country datasets?
  - They are outputs of the simulation it seems...
  - TexasDataCenterSolarModeling.pdf is a saved mathematica notebook output specifically calculating the output NorthTexas2.csv
- What is the plot?
  - X: load capex (\\$/MW), Y: Power system capex (\\$/MWh-load over 10 years)
- I'd need to find the data used for the other cities...
  - It's all here: https://www.nrel.gov/grid/solar-power-data

In [2]:
import os

In [5]:
os.getcwd()

'/Users/alecchen/Documents/Code/solar-modeling'

In [7]:
os.listdir('data')

['TexasDataCenterSolarModeling.pdf',
 'Britain.csv',
 'Solar modeling2.nb',
 'Actual_36.25_-102.95_2006_UPV_84MW_5_Min.csv',
 'drive-download-20250808T042704Z-1-001.zip',
 'tx-pv-2006.zip',
 'Maine.csv',
 'Arizona.csv',
 'California.csv',
 'NorthTexas2.csv',
 'Washington.csv']

In [36]:
import pandas as pd

In [2]:
az_df = pd.read_csv('data/Arizona.csv')
az_df.head()

,solar cost $/MW,battery cost $/MWh,load cost $/MW,arraysize (MW),battery size (MWh),load size (1 MW by definition),array cost $,battery cost $,load cost $ (all normalized to 1 MW),total power system cost $,total system cost $,total system cost per utilization,battery size relative to 1 MW array,load size relative to 1 MW array,annual battery utilization,annual load utilization
0,200000,200000,10000.000000,1.269183,0.0,1.0,253836.597598,0.0,10000.000000,253836.597598,263836.597598,1.107361e+06,0.0,0.787908,0.004119,0.238257
1,200000,200000,12589.254118,1.284408,0.0,1.0,256881.653255,0.0,12589.254118,256881.653255,269470.907373,1.118076e+06,0.0,0.778569,0.005099,0.241013
2,200000,200000,15848.931925,1.301215,0.0,1.0,260243.026274,0.0,15848.931925,260243.026274,276091.958198,1.131403e+06,0.0,0.768512,0.006431,0.244026
3,200000,200000,19952.623150,1.320610,0.0,1.0,264121.956898,0.0,19952.623150,264121.956898,284074.580048,1.147972e+06,0.0,0.757226,0.008048,0.247458
4,200000,200000,25118.864315,1.339795,0.0,1.0,267958.975204,0.0,25118.864315,267958.975204,293077.839519,1.168544e+06,0.0,0.746383,0.010008,0.250806


In [7]:
ca_df = pd.read_csv('data/California.csv')
ca_df.head()

,solar cost $/MW,battery cost $/MWh,load cost $/MW,arraysize (MW),battery size (MWh),load size (1 MW by definition),array cost $,battery cost $,load cost $ (all normalized to 1 MW),total power system cost $,total system cost $,total system cost per utilization,battery size relative to 1 MW array,load size relative to 1 MW array,annual battery utilization,annual load utilization
0,200000,200000,10000.000000,1.265840,0.0,1.0,253168.072084,0.0,10000.000000,253168.072084,263168.072084,1.051619e+06,0.0,0.789989,0.004271,0.250250
1,200000,200000,12589.254118,1.279195,0.0,1.0,255839.002405,0.0,12589.254118,255839.002405,268428.256523,1.061833e+06,0.0,0.781742,0.005146,0.252797
2,200000,200000,15848.931925,1.295417,0.0,1.0,259083.344454,0.0,15848.931925,259083.344454,274932.276378,1.074531e+06,0.0,0.771952,0.006621,0.255863
3,200000,200000,19952.623150,1.311187,0.0,1.0,262237.449740,0.0,19952.623150,262237.449740,282190.072890,1.090356e+06,0.0,0.762668,0.008219,0.258806
4,200000,200000,25118.864315,1.335469,0.0,1.0,267093.781284,0.0,25118.864315,267093.781284,292212.645599,1.109994e+06,0.0,0.748801,0.011168,0.263256


In [12]:
actual_df = pd.read_csv('data/Actual_36.25_-102.95_2006_UPV_84MW_5_Min.csv')
print(len(actual_df))
actual_df.head()

105120


,LocalTime,Power(MW)
0,01/01/06 00:00,0.0
1,01/01/06 00:05,0.0
2,01/01/06 00:10,0.0
3,01/01/06 00:15,0.0
4,01/01/06 00:20,0.0


In [11]:
actual_df = pd.read_csv('HA4_34.05_-102.05_2006_UPV_151MW_60_Min.csv')
actual_df.head()

,LocalTime,Power(MW)
0,01/01/06 00:00,0.0
1,01/01/06 01:00,0.0
2,01/01/06 02:00,0.0
3,01/01/06 03:00,0.0
4,01/01/06 04:00,0.0


### Download

In [49]:
import os
import requests

DATA_DIR = "data"

eastern_states = ['al', 'ak', 'ct', 'de', 'fl', 'ga', 'il', 'in', 'ia', 'ks', 'ky', 'la', 'me', 'md', 'ma', 'mi', 'mn', 'ms', 'mo', 'mt', 'ne', 'nh', 'nj', 'nm-east', 'ny', 'nc', 'oh', 'ok', 'pa', 'ri', 'sc', 'sd', 'tn', 'tx-east', 'vt', 'va', 'wv', 'wi']
western_states = ['az', 'ca', 'co', 'id', 'mt', 'nv', 'nm', 'or', 'sd', 'tx', 'ut', 'wa', 'wy']

def state_solar_zip_filename(state: str) -> str:
    return f"{state}-pv-2006.zip"

def state_download_url(state: str) -> str:
    return f"https://www.nrel.gov/docs/libraries/grid/{state_solar_zip_filename(state)}"

def state_data_dir(directory: str, state: str) -> str:
    return f"{directory}/{state}"

def download_state_solar_data(directory: str, state: str):
    os.makedirs(directory, exist_ok=True)
    url = state_download_url(state)
    extract_to = state_data_dir(directory, state)
    os.makedirs(extract_to, exist_ok=True)
    r = requests.get(url)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(extract_to)
    files = os.listdir(extract_to)
    preview_count = 5
    print("Downloaded:", ", ".join(files[:preview_count]), f"\n...{len(files)} total")

def download_all_solar_data(directory: str):
    for state in eastern_states + western_states:
        download_state_solar_data(directory, state)

In [51]:
download_state_solar_data(DATA_DIR, "az")

Downloaded: HA4_32.55_-112.05_2006_UPV_100MW_60_Min.csv, Actual_33.45_-112.15_2006_DPV_103MW_5_Min.csv, HA4_33.85_-112.55_2006_UPV_28MW_60_Min.csv, Actual_34.65_-114.15_2006_UPV_150MW_5_Min.csv, Actual_32.85_-112.25_2006_UPV_50MW_5_Min.csv 
...513 total


In [20]:
download_all_solar_data(DATA_DIR)

Downloaded: HA4_32.55_-112.05_2006_UPV_100MW_60_Min.csv, Actual_33.45_-112.15_2006_DPV_103MW_5_Min.csv, HA4_33.85_-112.55_2006_UPV_28MW_60_Min.csv, Actual_34.65_-114.15_2006_UPV_150MW_5_Min.csv, Actual_32.85_-112.25_2006_UPV_50MW_5_Min.csv ...513 total


In [59]:
from dataclasses import dataclass
from enum import Enum
import re

class DataType(str, Enum):
    ACTUAL = "Actual" # Real power output
    DA = "DA" # Day ahead forecast
    HA4 = "HA4" # 4 hour ahead forecast

class PvType(str, Enum):
    UPV = "UPV" # Utility scale PV
    DPV = "DPV" # Distributed PV

def solar_filename(data_type: DataType, latitude: float, longitude: float, weather_year: int, pv_type: PvType, capacity_mw: int, time_interval_min: int) -> str:
    return f"{data_type}_{round(latitude, 2)}_{round(longitude, 2)}_{weather_year}_{pv_type}_{capacity_mw}MW_{time_interval_min}_Min.csv"

@dataclass(frozen=True)
class Dataset:
    data_type: DataType
    latitude: float
    longitude: float
    weather_year: int
    pv_type: PvType
    capacity_mw: int
    time_interval_min: int

    def filename(self) -> str:
        return solar_filename(**self.__dict__)

    @classmethod
    def from_filename(cls, filename: str) -> "Dataset":
        name = filename.removesuffix(".csv")
        parts = name.split("_")

        if len(parts) != 8:  # 8 because of "5_Min" being two parts
            raise ValueError(f"Unexpected filename format: {filename}")

        data_type_str, lat_str, lon_str, year_str, pv_type_str, capacity_str, interval_val, interval_label = parts

        if interval_label != "Min":
            raise ValueError(f"Expected 'Min' at the end of interval, got '{interval_label}'")

        capacity_mw = int(re.match(r"(\d+)MW", capacity_str).group(1))
        interval_min = int(interval_val)

        return cls(
            data_type=DataType(data_type_str),
            latitude=float(lat_str),
            longitude=float(lon_str),
            weather_year=int(year_str),
            pv_type=PvType(pv_type_str),
            capacity_mw=capacity_mw,
            time_interval_min=interval_min
        )

example_filename = "Actual_33.45_-112.15_2006_DPV_103MW_5_Min.csv"
dataset = Dataset.from_filename(example_filename)
assert dataset.data_type == DataType.ACTUAL
assert dataset.latitude == 33.45
assert dataset.longitude == -112.15
assert dataset.weather_year == 2006
assert dataset.capacity_mw == 103
assert dataset.time_interval_min == 5
assert dataset.filename() == example_filename

### Exploration

In [37]:
import pandas as pd

In [32]:
AZ_DATA_DIR = f"{DATA_DIR}/az"
files = sorted(os.listdir(AZ_DATA_DIR))
len(files)

513

In [33]:
ACTUAL_PREFIX = "Actual"
DA_PREFIX = "DA"
HA4_PREFIX = "HA4"

actual_files = [file for file in files if file.startswith(ACTUAL_PREFIX)]
da_files = [file for file in files if file.startswith(DA_PREFIX)]
ha4_files = [file for file in files if file.startswith(HA4_PREFIX)]
len(actual_files) + len(da_files) + len(ha4_files)

513

In [40]:
actual_file = actual_files[0]
actual_df = pd.read_csv(f"{AZ_DATA_DIR}/{actual_file}")
actual_df.head()

,LocalTime,Power(MW)
0,01/01/06 00:00,0.0
1,01/01/06 00:05,0.0
2,01/01/06 00:10,0.0
3,01/01/06 00:15,0.0
4,01/01/06 00:20,0.0


In [42]:
da_file = da_files[0]
da_df = pd.read_csv(f"{AZ_DATA_DIR}/{da_file}")
da_df.head()

,LocalTime,Power(MW)
0,01/01/06 00:00,0.0
1,01/01/06 01:00,0.0
2,01/01/06 02:00,0.0
3,01/01/06 03:00,0.0
4,01/01/06 04:00,0.0


In [43]:
ha4_file = ha4_files[0]
ha4_df = pd.read_csv(f"{AZ_DATA_DIR}/{ha4_file}")
ha4_df.head()

,LocalTime,Power(MW)
0,01/01/06 00:00,0.0
1,01/01/06 01:00,0.0
2,01/01/06 02:00,0.0
3,01/01/06 03:00,0.0
4,01/01/06 04:00,0.0


In [44]:
POWER_COL = "Power(MW)"

In [45]:
actual_df[POWER_COL].describe()

count    105120.000000
mean         18.797293
std          25.987606
min           0.000000
25%           0.000000
50%           0.000000
75%          40.200000
max          94.800000
Name: Power(MW), dtype: float64

In [46]:
da_df[POWER_COL].describe()

count    8760.000000
mean       19.294064
std        26.597566
min         0.000000
25%         0.000000
50%         0.000000
75%        41.600000
max        86.000000
Name: Power(MW), dtype: float64

In [47]:
ha4_df[POWER_COL].describe()

count    8760.000000
mean       18.767283
std        25.660718
min         0.000000
25%         0.000000
50%         0.000000
75%        39.900000
max        84.700000
Name: Power(MW), dtype: float64